As of 2021-02-27. I have plotted a subset of sessions based on the amount of masking and cells they have. I then looked through each of these plots and determiend which sessions/neurons are best. They are stored on git under figures/2shit, 4shit, 6shit, all hits. See [here](https://docs.google.com/spreadsheets/d/14vj2YuSKi9pSUvye8D5rA6VBCqUm9ezUbh8kaTurJ1I/edit#gid=1622653460) for more info.

In [1]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
# stored one repo up in my fork of Spykes
from spykes.spykes.plot.neurovis import NeuroVis
from io_utils import *
from plotting_utils import * 

# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

In [6]:
# pick out sessions/neurons
sess_names = ['data_sdc_20190902_145404_fromSD', 'W122_08_06_2019_1_fromSD', 'W122_08_07_2019_1_fromSD',
             'W122_08_31_2019_1_fromSD', 'W122_09_09_2019_1_fromSD', 'W122_09_21_2019_1_fromSD']

sess_neurons = [[0,1,3], [0,2,3], [0,3], [1,2,3], [0,1], [1]]

# base paths
base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'
fig_save_path = os.path.join(os.getcwd(), 'figures', 'neurovis')


# create specific paths
sess_paths = []
beh_paths = []
spks_paths = []

for sess in range(len(sess_names)):
    sess_path = os.path.join(base_path, sess_names[sess])
    sess_paths.append(sess_path)
    
    beh_paths.append(os.path.join(sess_path, beh_mat))
    spks_paths.append(os.path.join(sess_path, spks_mat))

In [15]:
beh_dfs = []
spks_dicts = []

for sess in range(len(sess_names)):
    beh_df, spks_dict = load_and_wrangle(beh_paths[sess], spks_paths[sess], overwrite=True)
    beh_dfs.append(beh_df)
    spks_dicts.append(spks_dict)

Okay, everything is loaded in, but do I want it loaded in? Maybe I want to write a wrapper functions that only load spk_dict information for the neruons of interest. It seems easier to filter here than later on...

For example, given a list of neuron numbers you're interested in, only that spk information will be loaded into the dict. I could write a function called 'selective_load_and_wrangle(beh_path, spks_path, selected_neurons). And it calls a new function called  selective_make_spks_dict(spks_info, selected_neurons. Rather than looping over all cells for a session and loading, it only loads the cells in the list passed in.